In [3]:
def update_RR_avg(typee, update_info, disease_GWAS):
    RR_avg=pd.read_excel('RR_avg.xlsx')
    print('RR_avg中需要更新的数据：', update_info)

    if typee==True: #199种疾病的情况，更新全部列
        for index, row in RR_avg.iterrows():
            if row[0]==disease_GWAS:
                if np.isnan(row['OR_score_sum']):
                    RR_avg.loc[index, 'OR_score_sum']=float(update_info[0])
                else:
                    RR_avg.loc[index, 'OR_score_sum']=RR_avg.loc[index, 'OR_score_sum']+float(update_info[0])
            
                if np.isnan(row['OR_count']):
                    RR_avg.loc[index, 'OR_count']=float(1)
                else:
                    RR_avg.loc[index, 'OR_count']=RR_avg.loc[index, 'OR_count']+float(1)

                if np.isnan(row['SC_score_sum']):
                    RR_avg.loc[index, 'SC_score_sum']=float(update_info[1])
                else:
                    RR_avg.loc[index, 'SC_score_sum']=RR_avg.loc[index, 'SC_score_sum']+float(update_info[1])
            
                if np.isnan(row['SC_count']):
                    RR_avg.loc[index, 'SC_count']=float(1)
                else:
                    RR_avg.loc[index, 'SC_count']=RR_avg.loc[index, 'SC_count']+float(1)

                if np.isnan(row['SD_score_sum']):
                    RR_avg.loc[index, 'SD_score_sum']=float(update_info[2])
                else:
                    RR_avg.loc[index, 'SD_score_sum']=RR_avg.loc[index, 'SD_score_sum']+float(update_info[2])
            
                if np.isnan(row['SD_count']):
                    RR_avg.loc[index, 'SD_count']=float(1)
                else:
                    RR_avg.loc[index, 'SD_count']=RR_avg.loc[index, 'SD_count']+float(1)
                print('RR_avg文件更新的这一行信息为: ', RR_avg.loc[index])
                break

    else: #35种疾病，只更新SC_score
        for index, row in RR_avg.iterrows():
            if row[0]==disease_GWAS:
                if np.isnan(row[3]):
                    RR_avg.loc[index, 'SC_score_sum']=float(update_info)
                else:
                    RR_avg.loc[index, 'SC_score_sum']=RR_avg.loc[index, 'SC_score_sum']+float(update_info)
            
                if np.isnan(row[4]):
                    RR_avg.loc[index, 'SC_count']=float(1)
                else:
                    RR_avg.loc[index, 'SC_count']=RR_avg.loc[index, 'SC_count']+float(1)
                print('RR_avg文件更新的这一行信息为: ', RR_avg.loc[index])
                break

    RR_avg.to_excel('RR_avg.xlsx')

# 导入用户数据

In [4]:
import pandas as pd
Test_Data = pd.read_excel('test_data.xlsx')
Test_Data.dropna(axis=0, how='any', inplace=True)
print(Test_Data.shape[0])
Test_Data.head()

528843


,1.CEL_call_code,2.CEL_call_code,3.CEL_call_code,dbSNP_RS_ID
0,C/C,C/C,C/C,rs913197212
1,G/G,G/G,G/G,rs775015331
2,C/C,C/C,C/C,rs771563844
3,C/C,C/C,C/C,rs202136839
4,C/C,C/C,C/C,rs776714576


In [5]:
#导入用户基因数据
df_user = pd.DataFrame(columns=['rs_ID', 'Allele_1', 'Allele_2'])
for index, row in Test_Data.iterrows():  
    #计数器
    if index%100000==0:
        print('格式转换进度：  ', str(float(index)/float(Test_Data.shape[0])*100)[:5], '%')
        
    #排除编码里不存在信息
    code = row['1.CEL_call_code']
    if '-' in code:  
        continue
    else:
        allele_1 = code.split('/')[0][0]
        allele_2 = code.split('/')[1][0]
        if (allele_1 in ['A', 'T', 'C', 'G']) & (allele_2 in ['A', 'T', 'C', 'G']):
            df_user=df_user.append({'rs_ID': row['dbSNP_RS_ID'], 'Allele_1': allele_1, 'Allele_2': allele_2}, ignore_index=True)
    
user_rs=df_user['rs_ID']#按照rs_ID排序(方便后续匹配查找)
user_rs_num = len(user_rs)
print('用户表里的信息总数：', user_rs_num)

格式转换进度：   0.0 %
格式转换进度：   18.90 %
格式转换进度：   37.81 %
格式转换进度：   56.72 %
格式转换进度：   75.63 %
格式转换进度：   94.54 %
格式转换进度：   113.4 %
用户表里的信息总数： 528375


In [6]:
#创建输出表格
Valid_Results=pd.DataFrame(columns=['Chinese', 'English', 'OR_score','SC_score','SD_score', 'SNP_info'])
Report_data=pd.DataFrame(columns=['Chinese', 'English', 'OR_level','SC_level','SD_level', 'SNP_info'])
SNP_info=[]
Num_Match_SNP=[]

# 199种带OR值的疾病数据

In [10]:
from math import log
import SimpleCount
import OddsRatio
import PRSsd
import pandas as pd
import numpy as np
import score2level

##导入数据
Disease_Avai=pd.read_csv('199_disease_name.csv', usecols=['English_GWAS'])['English_GWAS'].tolist()
Disease_Chinese=pd.read_csv('199_disease_name.csv', usecols=['Chinese'])['Chinese'].tolist()
Disease_Avai_test=Disease_Avai

##计算分数
print('-'*20+"  数据库1  "+'-'*20)
for disease_GWAS, index in zip(Disease_Avai_test, range(len(Disease_Avai_test))):
    Chinese_name=Disease_Chinese[index]
    print("{:=^50s}".format('当前疾病： '+ disease_GWAS))
    print('当前运行进度： ' , index+1, '/', len(Disease_Avai_test))

    #获取GWAS数据库中关于该疾病的信息
    simplified_database_GWAS=pd.read_csv('simplified_database_GWAS.csv', usecols=['English_GWAS','Risk_allele','OR'])
    Specific_GWAS=simplified_database_GWAS[simplified_database_GWAS['English_GWAS']==disease_GWAS].loc[:, ['Risk_allele','OR']]#特定疾病在GWAS库里有的SNP以及对应OR值
    Specific_GWAS=Specific_GWAS.reset_index(drop=True)#重置index
    GWAS_rs = sorted(set([str(x).split('-')[0] for x in Specific_GWAS['Risk_allele']]))#提取GWAS中的SNP（去重）
    num_GWAS_rs = len(GWAS_rs)
    print('在GWAS库中该疾病有效的风险SNP个数： ', num_GWAS_rs)
    if len(Specific_GWAS)==0:
        print('err1：GWAS数据库中无SNP信息')
        Err_info='err1'
        continue

    #匹配GWAS中与用户基因文件中重合的SNP
    user_rs_index = []  #记录匹配上SNP在user表中的序号
    matched_rs = [] #记录匹配上的SNP
    for rs in GWAS_rs:
        for index, user_rs in zip(range(len(df_user['rs_ID'])), df_user['rs_ID']):
            if rs==user_rs:
                matched_rs.append(rs)
                user_rs_index.append(index)

    num_Match_rs=len(matched_rs)
    print("能匹配上的SNP个数：  ", num_Match_rs)
    if num_Match_rs==0:
        print('err2：无匹配上的SNP')
        Err_info='err2'
        continue
    print(matched_rs)  # 能配对上的SNP

    #建立权重字典：Dict_OR & Dict_SC （建立字典Dict存储每个SNP各风险等位基因的信息）(目前三种模型可以使用)
    #####   Dict_OR
    Dict_OR = dict() 
    #风险基因&OR值存入字典Dict(不包含用户不提供的SNP位点信息)
    #Dict_OR={'rs9592461': [['A', [7.062, 1.025]]], ['G', [1.021451]]], 'rs645592': [['T', [5.571]]]}
    for index, row in Specific_GWAS.iterrows():
        rs = str(row['Risk_allele']).split('-')[0]
        AGCT = str(row['Risk_allele']).split('-')[1][-1]
        if rs in matched_rs:
            if rs in Dict_OR : #rs在字典中存在
                if AGCT not in [i[0] for i in Dict_OR[rs]]:  #但AGCT尚未存在时
                    Dict_OR[rs].append([AGCT,[float(row['OR'])]])
                else:
                    for i in Dict_OR[rs]:
                        if i[0]==AGCT:
                            i[1].append(float(row['OR']))
            elif not(row['OR'] is None):
                Dict_OR[rs]=[[AGCT,[float(row['OR'])]]]

    #根据字典Dict计算各SNP等位风险基因对应的风险权重
    for key, value in Dict_OR.items():
        #value,eg.:[['A', [7.062, 1.025]]], ['G', [1.021451]]]
        n = len(value)
        if n > 0:
            for x in value:
                #x,eg.:['A', [7.062, 1.025]]
                AGCT = x[0]
                OR_list = x[1]
                OR_num=len(x[1])
                OR=sum(OR_list)/OR_num
                OR_log=log(OR)
                x[1]=OR_log
                #x,eg.:['A', 0.3]
            Dict_OR[key]=value
    print('使用到的位点信息： ', Dict_OR)  #{'rs1656369': [['A', 1.76541],['G', 1.76541549]], 'rs10811965': [['T', 1.7279318]]}


    #####   Dict_SC
    Dict_SC = dict() 
    for index, row in Specific_GWAS.iterrows():
        rs = str(row['Risk_allele']).split('-')[0]
        AGCT = str(row['Risk_allele']).split('-')[1][-1]
        if rs in matched_rs and not(row['OR'] is None):
            if rs in Dict_SC and AGCT not in Dict_SC[rs]:
                Dict_SC[rs].append(AGCT)
            else:
                Dict_SC[rs]=[AGCT] 
    print(Dict_SC) #{'rs1234:['A','G'], 'rs1235':['C']}


    #运行模型
    OR_score= OddsRatio.Method1_OR(Dict_OR, df_user, user_rs_index)
    SC_score= SimpleCount.Method2_SC(Dict_SC, df_user, user_rs_index)
    SD_score= PRSsd.Method3_PRSsd(disease_GWAS, OR_score)

    #更新分数记录表格
    update_RR_avg(True, [OR_score, SC_score, SD_score], disease_GWAS)
    Valid_Results=Valid_Results.append({'Chinese': Chinese_name, 'English': disease_GWAS,
    'OR_score': OR_score, 'SC_score': SC_score, 'SD_score': SD_score, 'SNP_info': Dict_OR}, ignore_index=True)
    Report_data=Report_data.append({'Chinese': Chinese_name, 'English': disease_GWAS,
    'OR_level': score2level.score2level(disease_GWAS, OR_score, 'OR'), 'SC_level': score2level.score2level(disease_GWAS, SC_score, 'SC'),\
         'SD_level': score2level.score2level(disease_GWAS, SD_score, 'SD'), 'SNP_info': Dict_OR}, ignore_index=True)




--------------------  数据库1  --------------------
=============当前疾病： Oral cavity cancer=============
当前运行进度：  1 / 199
在GWAS库中该疾病有效的风险SNP个数：  10
能匹配上的SNP个数：   0
err2：无匹配上的SNP
==================当前疾病： Melanoma==================
当前运行进度：  2 / 199
在GWAS库中该疾病有效的风险SNP个数：  60
能匹配上的SNP个数：   4
['rs1126809', 'rs1800407', 'rs401681', 'rs4408545']
使用到的位点信息：  {'rs401681': [['A', 0.1823215567939546]], 'rs4408545': [['C', 0.19589567682107248]], 'rs1800407': [['T', 0.1679453796494474]], 'rs1126809': [['A', 0.16605497033000804]]}
{'rs401681': ['A'], 'rs4408545': ['C'], 'rs1800407': ['T'], 'rs1126809': ['A']}
PRSsd表格中的数据： [0.2750503235716379, 0.1375251617858189, 0.1375251617858189, 0.1375251617858189, 0.2750503235716379, 0.2750503235716379, 0.2750503235716379, 0.2750503235716379, 0.1375251617858189, 0.1375251617858189, 0.2750503235716379, 0.2750503235716379, 0.2750503235716379, 0.2750503235716379, 0.0, 0.1375251617858189, 0.2750503235716379, 0.1375251617858189, 0.2750503235716379, 0.2750503235716379, 0.137

PRSsd表格中的数据： [0.1573242027540006, 0.447867276518377, 0.4072221509571823, 0.1573242027540006, 0.4632414684718946, 0.2506625085658215, 0.2572331129558948, 0.2153793887233607, 0.3525501947371359, 0.3404204699096053, 0.2572331129558948, 0.330605398393545, 0.2726073049094124, 0.2791779092994857, 0.1166790771928059, 0.3272221686442313, 0.1946431910511091, 0.2745860808788326, 0.1013048852392883, 0.1726983947075183, 0.1726983947075183, 0.1726983947075183, 0.1573242027540006, 0.1726983947075183, 0.1726983947075183, 0.1726983947075183, 0.1726983947075183, 0.3404204699096053, 0.2572331129558948, 0.330605398393545, 0.2726073049094124, 0.2791779092994857, 0.1166790771928059, 0.3272221686442313, 0.1946431910511091, 0.2745860808788326, 0.1013048852392883, 0.1726983947075183, 0.1573242027540006, 0.1573242027540006, 0.447867276518377, 0.4072221509571823, 0.1573242027540006, 0.4632414684718946, 0.2506625085658215, 0.2572331129558948, 0.2153793887233607, 0.3525501947371359, 0.3404204699096053, 0.25723311

PRSsd表格中的数据： [0.3366207418667121, 0.168310370933356, 0.168310370933356, 0.0, 0.168310370933356, 0.168310370933356, 0.168310370933356, 0.168310370933356, 0.0, 0.168310370933356, 0.168310370933356, 0.0, 0.168310370933356, 0.0, 0.0, 0.0, 0.0, 0.0, 0.168310370933356, 0.168310370933356, 0.168310370933356, 0.168310370933356, 0.3366207418667121, 0.168310370933356, 0.168310370933356, 0.168310370933356, 0.168310370933356, 0.168310370933356, 0.168310370933356, 0.0, 0.168310370933356, 0.0, 0.0, 0.0, 0.0, 0.0, 0.168310370933356, 0.168310370933356, 0.3366207418667121, 0.3366207418667121, 0.168310370933356, 0.168310370933356, 0.0, 0.168310370933356, 0.168310370933356, 0.168310370933356, 0.168310370933356, 0.0, 0.168310370933356, 0.168310370933356, 0.0, 0.168310370933356, 0.0, 0.0, 0.0, 0.0, 0.0, 0.168310370933356, 0.168310370933356, 0.3366207418667121, 0.168310370933356, 0.168310370933356, 0.0, 0.168310370933356, 0.168310370933356, 0.168310370933356, 0.0]
RR_avg中需要更新的数据： [0.0, 0.0, 1.206807492370245

能匹配上的SNP个数：   2
['rs29232', 'rs3129055']
使用到的位点信息：  {'rs3129055': [['G', 0.412109650826833]], 'rs29232': [['A', 0.5128236264286637]]}
{'rs3129055': ['G'], 'rs29232': ['A']}
PRSsd表格中的数据： [0.4455560860018607, 0.0, 0.0, 0.2227780430009303, 0.0, 0.4455560860018607, 0.2227780430009303, 0.2227780430009303, 0.4455560860018607, 0.0, 0.2227780430009303, 0.0, 0.0, 0.0, 0.0, 0.2227780430009303, 0.0, 0.2227780430009303, 0.2227780430009303, 0.0, 0.0, 0.0, 0.4455560860018607, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2227780430009303, 0.0, 0.0, 0.0, 0.0, 0.2227780430009303, 0.0, 0.2227780430009303, 0.2227780430009303, 0.0, 0.4455560860018607, 0.4455560860018607, 0.0, 0.0, 0.2227780430009303, 0.0, 0.4455560860018607, 0.2227780430009303, 0.2227780430009303, 0.4455560860018607, 0.0, 0.2227780430009303, 0.0, 0.0, 0.0, 0.0, 0.2227780430009303, 0.0, 0.2227780430009303, 0.2227780430009303, 0.0, 0.4455560860018607, 0.0, 0.0, 0.2227780430009303, 0.0, 0.4455560860018607, 0.4455560860018607, 0.0]
RR_avg中需要更新的数据： [0.0, 0.0, 0

在GWAS库中该疾病有效的风险SNP个数：  0
err1：GWAS数据库中无SNP信息
==============当前疾病： Multiple myeloma==============
当前运行进度：  26 / 199
在GWAS库中该疾病有效的风险SNP个数：  51
能匹配上的SNP个数：   10
['rs1050976', 'rs10936599', 'rs10936600', 'rs13296848', 'rs2047077', 'rs2285803', 'rs28362345', 'rs34562254', 'rs7193541', 'rs877529']
使用到的位点信息：  {'rs2047077': [['G', 0.5068176023684519]], 'rs28362345': [['T', 0.2151113796169455]], 'rs13296848': [['C', 0.22314355131420976]], 'rs10936599': [['G', 0.22793206804600694]], 'rs2285803': [['A', 0.17563256864315796]], 'rs877529': [['A', 0.2053868297249507]], 'rs7193541': [['T', 0.12221763272424911]], 'rs10936600': [['A', 0.1823215567939546]], 'rs34562254': [['A', 0.26236426446749106]], 'rs1050976': [['T', 0.09531017980432493]]}
{'rs2047077': ['G'], 'rs28362345': ['T'], 'rs13296848': ['C'], 'rs10936599': ['G'], 'rs2285803': ['A'], 'rs877529': ['A'], 'rs7193541': ['T'], 'rs10936600': ['A'], 'rs34562254': ['A'], 'rs1050976': ['T']}
PRSsd表格中的数据： [0.1525396386129929, 0.2740351502442143, 0.29209

...

Group_avg:   0.4486111111111107
Group_avg:   4.47868306238097
============当前疾病： HIV-1 susceptibility============
当前运行进度：  152 / 199
在GWAS库中该疾病有效的风险SNP个数：  5
能匹配上的SNP个数：   2
['rs4418214', 'rs4878712']
使用到的位点信息：  {'rs4418214': [['C', 0.41871033485818504]], 'rs4878712': [['8', 0.494696241836107], ['A', 0.27763173659827955]]}
{'rs4418214': ['C'], 'rs4878712': ['8', 'A']}
PRSsd表格中的数据： [0.233100592566816, 0.116550296283408, 0.116550296283408, 0.116550296283408, 0.233100592566816, 0.116550296283408, 0.116550296283408, 0.233100592566816, 0.116550296283408, 0.233100592566816, 0.233100592566816, 0.4088759350495776, 0.116550296283408, 0.0, 0.4088759350495776, 0.116550296283408, 0.233100592566816, 0.0, 0.233100592566816, 0.116550296283408, 0.116550296283408, 0.233100592566816, 0.233100592566816, 0.4088759350495776, 0.116550296283408, 0.0, 0.4088759350495776, 0.116550296283408, 0.233100592566816, 0.0, 0.233100592566816, 0.116550296283408, 0.233100592566816, 0.233100592566816, 0.116550296283408, 0.1

能匹配上的SNP个数：   12
['rs10890311', 'rs11707641', 'rs12519165', 'rs1334576', 'rs17486278', 'rs2070600', 'rs4523894', 'rs62065216', 'rs721917', 'rs8034191', 'rs8042238', 'rs9525927']
使用到的位点信息：  {'rs8034191': [['C', 0.3364722366212129]], 'rs8042238': [['T', 0.23111172096338664]], 'rs17486278': [['C', 0.16551443847757333]], 'rs721917': [['G', 0.06765864847381486]], 'rs2070600': [['C', 0.20294084399669038]], 'rs12519165': [['A', 0.06765864847381486]], 'rs9525927': [['G', 0.0769610411361284]], 'rs1334576': [['A', 0.058268908123975824]], 'rs62065216': [['A', 0.058268908123975824]], 'rs10890311': [['T', 0.058268908123975824]], 'rs4523894': [['C', 0.11332868530700327]], 'rs11707641': [['T', 0.06765864847381486]]}
{'rs8034191': ['C'], 'rs8042238': ['T'], 'rs17486278': ['C'], 'rs721917': ['G'], 'rs2070600': ['C'], 'rs12519165': ['A'], 'rs9525927': ['G'], 'rs1334576': ['A'], 'rs62065216': ['A'], 'rs10890311': ['T'], 'rs4523894': ['C'], 'rs11707641': ['T']}
PRSsd表格中的数据： [0.4195498426517499, 0.68636246

PRSsd表格中的数据： [0.3855100592646188, 0.3072449703676905, 0.8072449703676906, 0.6927550296323094, 0.1927550296323094, 0.3855100592646188, 0.6927550296323094, 0.8072449703676906, 0.3855100592646188, 0.1927550296323094, 0.3855100592646188, 0.5, 0.6927550296323094, 0.1927550296323094, 0.1927550296323094, 0.614489940735381, 0.6927550296323094, 1.0, 1.0, 1.0, 1.0, 0.1927550296323094, 0.3855100592646188, 0.5, 0.6927550296323094, 0.1927550296323094, 0.1927550296323094, 0.614489940735381, 0.6927550296323094, 1.0, 1.0, 1.0, 0.3855100592646188, 0.3855100592646188, 0.3072449703676905, 0.8072449703676906, 0.6927550296323094, 0.1927550296323094, 0.3855100592646188, 0.6927550296323094, 0.8072449703676906, 0.3855100592646188, 0.1927550296323094, 0.3855100592646188, 0.5, 0.6927550296323094, 0.1927550296323094, 0.1927550296323094, 0.614489940735381, 0.6927550296323094, 1.0, 1.0, 1.0, 0.3855100592646188, 0.3072449703676905, 0.8072449703676906, 0.6927550296323094, 0.1927550296323094, 0.3855100592646188, 0.69

Group_avg:   0.4166666666666667
Group_avg:   1.2427542336540838
=================当前疾病： Hepatitis==================
当前运行进度：  185 / 199
在GWAS库中该疾病有效的风险SNP个数：  6
能匹配上的SNP个数：   1
['rs11025185']
使用到的位点信息：  {'rs11025185': [['A', 0.13976194237515863]]}
{'rs11025185': ['A']}
PRSsd表格中的数据： [0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0]
RR_avg中需要更新的数据： [0.0, 0.0, 0.44721359549995787]
RR_avg文件更新的这一行信息为:  English         Hepatitis
OR_score_sum          3.5
OR_count               60
SC_score_sum            5
SC_count               60
SD_score_sum      29.1989
SD_count               60
Name: 219, dtype: object
Group_avg:   0.058333333333333334
Group_avg:   0.08333333333333333
Group_avg:   0.48664767935446146
=============当前疾病： Gallstone disease=======

Group_avg:   1.5636838540252653
===========当前疾病： Osteoprotegerin levels===========
当前运行进度：  194 / 199
在GWAS库中该疾病有效的风险SNP个数：  0
err1：GWAS数据库中无SNP信息
=================当前疾病： Scoliosis==================
当前运行进度：  195 / 199
在GWAS库中该疾病有效的风险SNP个数：  0
err1：GWAS数据库中无SNP信息
当前疾病： Idiopathic osteonecrosis of the femoral head
当前运行进度：  196 / 199
在GWAS库中该疾病有效的风险SNP个数：  1
能匹配上的SNP个数：   0
err2：无匹配上的SNP
=============当前疾病： Rotator cuff tears=============
当前运行进度：  197 / 199
在GWAS库中该疾病有效的风险SNP个数：  0
err1：GWAS数据库中无SNP信息
======当前疾病： Temporomandibular joint disorder======
当前运行进度：  198 / 199
在GWAS库中该疾病有效的风险SNP个数：  7
能匹配上的SNP个数：   0
err2：无匹配上的SNP
===============当前疾病： nephroblastoma===============
当前运行进度：  199 / 199
在GWAS库中该疾病有效的风险SNP个数：  7
能匹配上的SNP个数：   3
['rs2495478', 'rs5955543', 'rs790356']
使用到的位点信息：  {'rs2495478': [['T', 0.3364722366212129]], 'rs790356': [['G', 0.2468600779315258]], 'rs5955543': [['G', 0.9707789171582248]]}
{'rs2495478': ['T'], 'rs790356': ['G'], 'rs5955543': ['G']}
PRSsd表格中的数据： [0.1588432493

# 导入35种不带OR值的疾病数据（目前只有SC一种模型）

In [ ]:

    print('-'*20+"  数据库2  "+'-'*20)
    ##导入数据
    Disease_Avai=pd.read_excel('35_disease_name.xlsx')['English'].tolist()
    Disease_Chinese=pd.read_excel('35_disease_name.xlsx')['Chinese'].tolist()
    Disease_Avai_test=Disease_Avai

    for disease_GWAS, index in zip(Disease_Avai_test, range(len(Disease_Avai_test))):
        print('当前疾病： ', disease_GWAS)
        print('当前运行进度： ' , index+1, '/', len(Disease_Avai_test))
        Chinese_name=Disease_Chinese[index]

        #获取GWAS数据库中关于该疾病的信息
        No_OR_database=pd.read_excel('35_disease_database.xlsx')
        No_OR_database=No_OR_database[No_OR_database['English']==disease_GWAS].loc[:, ['Risk_allele']]#特定疾病在GWAS库里有的SNP以及对应OR值
        No_OR_database=No_OR_database.reset_index(drop=True)#重置index
        GWAS_rs = sorted(set([str(x).split('-')[0] for x in No_OR_database['Risk_allele']]))#提取GWAS中的SNP（去重）
        print('在GWAS库中该疾病有效的风险SNP个数： ', len(No_OR_database))
        if len(No_OR_database)==0:
            print('err1：GWAS数据库中无SNP信息')
            Err_info='err1'
            continue

        #匹配GWAS中与用户基因文件中重合的SNP
        user_rs_index = []  #记录匹配上SNP在user表中的序号
        matched_rs = [] #记录匹配上的SNP
        for rs in GWAS_rs:
            for index, user_rs in zip(range(len(df_user['rs_ID'])), df_user['rs_ID']):
                if rs==user_rs:
                    matched_rs.append(rs)
                    user_rs_index.append(index)

        Num_Match_rs=len(matched_rs)
        print("能匹配上的SNP个数：  ", Num_Match_rs)
        if Num_Match_rs==0:
            print('err2：无匹配上的SNP')
            Err_info='err2'
            continue
        print(matched_rs)  # 能配对上的SNP
    
        #####   Dict_SC
        Dict_SC = dict() 
        for index, row in No_OR_database.iterrows():
            rs = str(row['Risk_allele']).split('-')[0]
            AGCT = str(row['Risk_allele']).split('-')[1][-1]
            if rs in matched_rs :
                if rs in Dict_SC and AGCT not in Dict_SC[rs]:
                    Dict_SC[rs].append(AGCT)
                else:
                    Dict_SC[rs]=[AGCT] 
        print(Dict_SC) #{'rs1234:['A','G'], 'rs1235':['C']}

        #计算风险分数
        SC_score= SimpleCount.Method2_SC(Dict_SC, df_user, user_rs_index)
        #更新分数记录表格
        update_RR_avg(False, SC_score, disease_GWAS)       
        Valid_Results=Valid_Results.append({'Chinese': Disease_Chinese[index], 'English': disease_GWAS,
        'SC_score': SC_score, 'SNP_info': Dict_SC} , ignore_index=True)
        Report_data=Report_data.append({'Chinese': Chinese_name, 'English': disease_GWAS,
        'SC_level': score2level.score2level(disease_GWAS, SC_score, 'SC'), 'SNP_info': Dict_OR}, ignore_index=True)

# 将结果整理并输出至文件

In [ ]:
file_prefix=file_dir.split("/")[-1]
Valid_Results.to_excel('output_data/valid/'+file_prefix+'_valid.xlsx',index=False)
Report_data.to_excel('output_data/report_data/'+file_prefix+'_report_data.xlsx',index=False)